<h3>Keras preprocessing layers</h3>
<p>In keras, preprocessing layers API will allow developers to build keras-native input processing pipelines. These pipelines can be used as independent preprocessing code in non-keras models or workflows. We can combine it directly with keras models and exported as part of keras SavedModel. Using Keras preprocessing layers, we can build and export models that are truly end-to-end, which means models that accept raw images or raw structured data as input, model that handle feature normalization or feature value indexing by own.</p>

<h2>Available preprocessing layers</h2>
<h4>Core processing layers</h4>
<ul><li><b>TextVectorization</b> layer, which will turn raw strings into an encoded representations that can be read by an embedding or dense layer.</li>
<li><b>Normalization</b> layer, it will performs feature-wise normalize of input features.</li></ul>
<h4>Structured data preprocessing layers (for structured data encoding and feature engineering).</h4>
<ul><li><b>CategoryEncoding</b> layer, will turn integer categorical features into one-hot, multi-hot or count dense representations.</li>
<li><b>Hashing</b> layer, will performs categorical feature hashing, also known as hashing trick.</li>
<li><b>Discretization</b> layer, will turns continuous numercal features into integer categorical features.</li>
<li><b>StringLookup</b> layer, will turns string categorical values to an encoded representations that can be read by an Embedding or Dense layer.</li>
<li><b>IntegerLookup</b> layer, will turns integer categorical values to an encoded representations that can be read by an Embedding or Dense layer.</li>
<li><b>CategoryCrossing</b> layer, it will combine categorical features into co-occurance features. Example, if the feature values are 'x' and 'y', it can provide with the combination feature 'x and y are present at the same time'.</li></ul>
<h4>Image preprocessing layers</h4>
<p>These layers standardize the inputs of an image model.</p>
<ul><li><b>Resizing</b> layer, will resize batch of images to a target size.</li>
<li><b>Rescaling</b> layer, will rescales and offsets the value of a batch of image(example, go from inputs in the [0, 255] range to inputs in the range [0, 1]).</li>
<li><b>CenterCrop</b> layer, will returns a center crop of batch of images.</li>
</ul>
<h4> Image data augmentation layers</h4>
<p>These layers apply random augmentation transforms to a batch of images. These layers are only active during training.</p>
<ul><li><b>RandomCrop</b> layer</li>
<li><b>RandomFlip</b> layer</li>
<li><b>RandomTranslation</b> layer</li>
<li><b>RandomRotation</b> layer</li>
<li><b>RandomZoom</b> layer</li>
<li><b>RandomHeight</b> layer</li>
<li><b>RandomWidth</b> layer</li></ul>

<h2> The adapt() method</h2>
<p>There are some preprocessing layers which have an internal state that must be computed based on a sample of the training data. The list of stateful preprocessing layers are:</p>
<ul><li><b>TextVectorization</b>: it holds a mapping between tokens and integer indices.</li>
<li><b>StringLookup</b> and <b>IntegerLookup</b>: it holds a mapping between integer/string values to integer indices.</li>
<li><b>Normalization</b>: it holds the mean and standard deviation of the features.</li>
<li><b>Discretization</b>: it holds information about value bucket boundaries.</li></ul>
<p>These layers are non-trainable. Their state must be set before training, this step is called 'adaption'. We can set the state of preprocessing layer by exposing it to training data by the <b>adapt()</b> method. </p>

In [2]:
# Importing required libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

# Creating data using numpy array
data = np.array([[0.1, 0.2, 0.3], [0.8, 0.9, 1.0], [1.5, 1.6, 1.7],])

# Applying normalization state to the preprocessing layer
layer = preprocessing.Normalization()

# Apllying the preprocessing layer state to the data 
layer.adapt(data)
normalized_data = layer(data)

#Displaying the mean and standard deviation of normalized data
print("Features mean: %.2f" % (normalized_data.numpy().mean()))
print("Features std: %.2f" % (normalized_data.numpy().std()))

Features mean: -0.00
Features std: 1.00


<p>The adopt() method takes either a Numpy array or a tf.data.Dataset object. In case of StringLookup and TextVectorization, we can pass a list of integers.</p>

In [3]:
# Creating the list of string
data = [
    "Boat Rockerz 300 wireless Neckband with ASAP cahrge",
    "Amazon Brand - Solimo cotton Buds -200 sticks.",
    "Amazon Brand - Solimo Plastic Lunch Box set 290 ml",
    "Maybeline New York color sensational creamy mate lipstick",
    "Life digital laptop 15-6-inch, citi bank offer",
    "Kwality Choco Flakes, made with whole wheat, maida",
    "AmbiTech High Accuracy Fingerprint Pulse Oximeter",
    "Gillette Classic Regular pre shave Foam 418 kg ",
]

# Adiing Textvectorization to the preprocessing layer
layer = preprocessing.TextVectorization()

# Applying the prepocessing layer state to the data
layer.adapt(data)
vectorized_text = layer(data)

# Displaying the vectorized text 
print(vectorized_text)

tf.Tensor(
[[49 14 55  7 22  2 51 46  0]
 [ 5  4  3 41 47 57 10  0  0]
 [ 5  4  3 18 28 48 12 56 23]
 [24 21  6 42 13 40 25 29  0]
 [30 39 31 58 44 50 20  0  0]
 [32 45 37 27  2  8  9 26  0]
 [52 34 53 38 16 19  0  0  0]
 [35 43 15 17 11 36 54 33  0]], shape=(8, 9), dtype=int64)


<p>Adaption layers always expose an option to directly set state via constructor arguments or weight assignment. If the intended state values are known at layer construction time or which are calculated outside the adapt() method call, they can be set without relying on the layer's internal computation. For instance, if external vocabulary files for the TextVectorization, StringLookup and IntegerLookup layers already exists, those can be loaded into lookup tables directly by passing a path to the vocabulary file in the layer's constructor arguments. Here we instantiate a StringLookup layer with precomputed vocabulary.</p>

In [4]:
# Creating a vocabulary
vocab = ["a", "b", "c", "d"]

# Creating the data by tf.constant
data = tf.constant([["a", "c", "d"], ["d", "z", "b"]])

#Adding StrigLookup preprocessing layer with already created vocab
layer = preprocessing.StringLookup(vocabulary=vocab)

#Applying the preprocessing layer to the data
vectorized_data = layer(data)
print(vectorized_data)

tf.Tensor(
[[2 4 5]
 [5 1 3]], shape=(2, 3), dtype=int64)


<h2>Preprocessing data before the model or inside the model.</h2>
<p>There are two ways for using the preprocessing layer.<br>
<b>Option-1</b> Make them part of the model.</p>



```
# Creating the input layer
inputs = keras.Input(shape=input_shape)

#Creating the preprocessing layer
x = preprocessing_layer(inputs)

#Creating the output layer
outputs = rest_of_the_model(x)

#Creating the keras model with inputs and outputs
model = keras.Model(inputs, outputs)
```



<p>With this option, preprocessing will happen on device, synchronously with the rest of the model execution, it means, it will benifit from GPU acceleration. If we're training on GPU, this is the best option for the normalization layer, and for all image preprocessing and data augmentation layers.<br>
<b>Option-2: </b>Apply it to your tf.data.Dataset, so as to obtain a dataset that yields batches of preprocessed data.</p> 



```
dataset = dataset.map(
 lambda x, y: (preprocessing_layer(x), y))```

